In [ ]:
pip install timm


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pip install git+https://github.com/rwightman/pytorch-image-models.git

  Cloning https://github.com/rwightman/pytorch-image-models.git to /tmp/pip-req-build-h750dqme
  Running command git clone -q https://github.com/rwightman/pytorch-image-models.git /tmp/pip-req-build-h750dqme


In [ ]:
import timm

m = timm.create_model('mobilenetv3_large_100', pretrained=True)
m.eval()

MobileNetV3(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): Hardswish()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [ ]:
import timm
model = timm.create_model('inception_v4', pretrained=True)
model.eval()

InceptionV4(
  (features): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (2): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (3): Mixed3a(
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv): BasicConv2d(
        (conv): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, mom

In [ ]:
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open(filename).convert('RGB')
tensor = transform(img).unsqueeze(0) # transform and add batch dimension

In [ ]:
import torch
with torch.no_grad():
    out = model(tensor)
probabilities = torch.nn.functional.softmax(out[0], dim=0)
print(probabilities.shape)
# prints: torch.Size([1000])

torch.Size([1000])


In [ ]:
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Print top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.9300336837768555
white wolf 0.0042124041356146336
Arctic fox 0.00265918648801744
Pomeranian 0.0016048167599365115
keeshond 0.0015494012041017413


In [ ]:
model = timm.create_model('inception_v4', pretrained=True, num_classes=8)

In [ ]:


from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Flatten, Dropout, Lambda
from keras.layers.embeddings import Embedding
import tensorflow as tf

from keras import models

batch_size = 50    

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '//content/drive/MyDrive/ihds_resized',
    labels='inferred',
    label_mode = "categorical",
     #class_names= ['Eraser','Mats','Handbags','Earbuds','Rubik's Cube']
    color_mode='rgb',
    batch_size=batch_size,
    
    shuffle = True,
    seed = 123,
    validation_split=0.2,
    subset="training"
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/',
    labels='inferred',
    label_mode = "categorical",
     #class_names= ['Eraser','Mats','Handbags','Earbuds','Rubik's Cube']
    color_mode='rgb',
    batch_size=batch_size,
    
    shuffle = True,
    seed = 123,
    validation_split=0.2,
    subset="validation"
)

Found 4287 files belonging to 2 classes.
Using 3430 files for training.
Found 5762 files belonging to 6 classes.
Using 1152 files for validation.


In [ ]:
import cv2
import torch
import torchvision
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [ ]:
if torch.cuda.is_available():
    device=torch.device("cuda:0")
    print("Running on GPU")
else:
    device=torch.device("cpu")
    print("Running on CPU")

Running on CPU


In [ ]:
!pip install timm
!pip install git+https://github.com/rwightman/pytorch-image-models.git

  Cloning https://github.com/rwightman/pytorch-image-models.git to /tmp/pip-req-build-3ph67353
  Running command git clone -q https://github.com/rwightman/pytorch-image-models.git /tmp/pip-req-build-3ph67353


In [ ]:
import timm
model = timm.create_model('inception_v4', pretrained=False, num_classes=8).to(device)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5), (0.5))])

In [ ]:
train_data = datasets.ImageFolder("/content/drive/MyDrive/EIS/Dataset",       
                    transform=transform)
test_data = datasets.ImageFolder("/content/drive/MyDrive/EIS/Dataset",       
                    transform=transform)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)
epochs = 5

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    # print(preds,labels)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
for e in range(epochs):
    train_loss=0
    testing_loss=0
    test_acc=0
    train_acc=0
    for images,labels in tqdm(trainloader):
        model.train()
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output,labels)
        train_loss += loss.item() 
        train_acc = accuracy(output,labels)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        for images,labels in tqdm(testloader):
            model.eval()
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = criterion(output,labels)
            testing_loss += loss.item()
            test_acc = accuracy(output,labels)

    print('Epoch:{} , Loss:{:.4f}, Testing Loss:{:.4f}, Training Accuracy:{:.4f}, Testing Accuracy:{:.4f}'.format(e,train_loss,testing_loss,train_acc*100.,test_acc*100.))

  0%|          | 0/24 [00:02<?, ?it/s]


RuntimeError: ignored

In [ ]:
EPOCH = 29
PATH = "model2.pt"
LOSS = 36.3433

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)